# Tarea #2: Representación vectorial de palabras con Word2Vec y FastText usando Gensim

Realziado por:
- Jose Luis Hincapie Bucheli (2125340)
- Sebastián Idrobo Avirama (2122637)
- Paul Rodrigo Rojas Guerrero (2127891)
---

# Creación de listas de sentencias

In [ ]:
!pip install datasets spacy

In [ ]:
from datasets import load_dataset

In [ ]:
raw_text = load_dataset('large_spanish_corpus', name='DGT', split='train', streaming=True)

## Tokenización

In [ ]:
import spacy
from tqdm import tqdm
import time

!python -m spacy download es_core_news_sm

In [ ]:
#Correr antes de hacer la limpieza
nlp = spacy.load('es_core_news_sm')

In [ ]:
print(time.ctime(time.time()))

with open('output.txt', 'w') as output_file:
    for sent in tqdm(raw_text):
        sent = sent['text']
        doc = nlp(sent)
    
        for token in doc:
            output_file.write(f'{token}, ')
    
        output_file.write('\n')

print(time.ctime(time.time()))

## Depuración

In [ ]:
# Carga de las palabras toquenizadas 
with open("output.txt","r",encoding="utf-8") as file:
    file_contents = file.read()
words = file_contents.split(", ")
print(words[1:200])

In [ ]:
import re, string

words_wn = []

with open('output2.txt', 'w',encoding="utf-8") as output_file:
    for w in tqdm(words):
        doc = nlp(w)
        # Verificar que sea una palabra, no sea un dígito y no sea una stop-word
        if len(doc) > 0 and not w.isdigit() and not doc[0].is_stop:
            w = re.sub(r'[0-9]','', w) # Depurar de dígitos
            re_punc = re.compile('[%s]' % re.escape(string.punctuation)) 
            w = re_punc.sub('', w) # Depurar de signos de puntuación
            w = re.sub("\!|\'|\?|\¿|\¡|\«|\»|\—","",w) # Depurar de símbolos especiales
            if w.strip(): # Si el token se convirtió en una cadena vacía o con espacios, no añadirla
                output_file.write(f'{w}, ')

In [ ]:
with open('output2.txt', 'r', encoding="utf-8") as file:
    lines = file.readlines()
    sentences = []
    for line in lines:
        words = line.strip().split(', ')
        sentences.append(words)

print(sentences[1:50])

# Representación Vectorial

## Construcción de modelos de FastText y Word2Vec

In [ ]:
# Inserte código aquí

## Construcción de vectores para palabras únicas

In [ ]:
# Inserte código aquí

## Visualización de vectores en plano cartesiano

In [ ]:
# Inserte código aquí